In [1]:
import pandas as pd
import sys
import math
from collections import defaultdict

infile='/Users/irffanalahi/Research/Research_code/gitignorefolder/target_panel/intro_exon_bp/intron_genecode_v36_modified.txt_canonical_2.txt'
outfile=infile+"_bpscaled"

origindf=pd.read_csv(infile,sep="\t")
indf=origindf.copy()
indf.head()

#hg38.knownCanonical.chrom_x  hg38.knownCanonical.chromStart_x  \
0                         chr1                             12227   
1                         chr1                             12721   
2                         chr1                             14501   
3                         chr1                             15038   
4                         chr1                             15947   

   hg38.knownCanonical.chromEnd_x hg38.knownCanonical.transcript  \
0                           12612              ENST00000456328.2   
1                           13220              ENST00000456328.2   
2                           15004              ENST00000488147.1   
3                           15795              ENST00000488147.1   
4                           16606              ENST00000488147.1   

   transcriptno direction #hg38.knownCanonical.chrom_y  \
0             0         +                         chr1   
1             1         +                         chr1   
2             0         -                         chr1   
3             1         -                         chr1   
4             2         -                         chr1   

   hg38.knownCanonical.chromStart_y  hg38.knownCanonical.chromEnd_y  \
0                             11868                           14409   
1                             11868                           14409   
2                             14403                           29570   
3                             14403                           29570   
4                             14403                           29570   

  hg38.kgXref.geneSymbol  
0                DDX11L1  
1                DDX11L1  
2                 WASH7P  
3                 WASH7P  
4                 WASH7P

In [2]:
indf.drop
indf[indf.duplicated(subset=['hg38.knownCanonical.transcript','transcriptno'])]

#hg38.knownCanonical.chrom_x  hg38.knownCanonical.chromStart_x  \
115763                         chrY                            253846   
115771                         chrY                            276394   
115772                         chrY                            281684   
115773                         chrY                            284314   
115774                         chrY                            288869   
...                             ...                               ...   
123686                         chrY                          57211620   
123691                         chrY                          57213125   
123692                         chrY                          57213357   
123693                         chrY                          57213602   
123694                         chrY                          57213964   

        hg38.knownCanonical.chromEnd_x hg38.knownCanonical.transcript  \
115763                          254936              ENST00000431238.7   
115771                          281481              ENST00000399012.6   
115772                          284166              ENST00000399012.6   
115773                          288732              ENST00000399012.6   
115774                          290647              ENST00000399012.6   
...                                ...                            ...   
123686                        57211760              ENST00000359512.8   
123691                        57213203              ENST00000507418.6   
123692                        57213525              ENST00000507418.6   
123693                        57213879              ENST00000507418.6   
123694                        57214349              ENST00000507418.6   

        transcriptno direction #hg38.knownCanonical.chrom_y  \
115763             0         +                         chrY   
115771             0         +                         chrY   
115772             1         +                         chrY   
115773             2         +                         chrY   
115774             3         +                         chrY   
...              ...       ...                          ...   
123686             8         +                         chrY   
123691             0         -                         chrY   
123692             1         -                         chrY   
123693             2         -                         chrY   
123694             3         -                         chrY   

        hg38.knownCanonical.chromStart_y  hg38.knownCanonical.chromEnd_y  \
115763                            253742                          255091   
115771                            276321                          303353   
115772                            276321                          303353   
115773                            276321                          303353   
115774                            276321                          303353   
...                                  ...                             ...   
123686                          57207480                        57211894   
123691                          57212183                        57214397   
123692                          57212183                        57214397   
123693                          57212183                        57214397   
123694                          57212183                        57214397   

       hg38.kgXref.geneSymbol  
115763            AL954722.1,  
115771                PLCXD1,  
115772                PLCXD1,  
115773                PLCXD1,  
115774                PLCXD1,  
...                       ...  
123686                WASH6P,  
123691              DDX11L16,  
123692              DDX11L16,  
123693              DDX11L16,  
123694              DDX11L16,  

[163 rows x 10 columns]

In [3]:
indf['feature_size']=indf['hg38.knownCanonical.chromEnd_x']-indf['hg38.knownCanonical.chromStart_x']
indf['totalbp_forcorrespondingTranscript']=-1
indf['bp_percentage']=-1
indf['bp_inscaled_block']=-1

indf['startingtranNumber']=-1

indf.head()

#hg38.knownCanonical.chrom_x  hg38.knownCanonical.chromStart_x  \
0                         chr1                             12227   
1                         chr1                             12721   
2                         chr1                             14501   
3                         chr1                             15038   
4                         chr1                             15947   

   hg38.knownCanonical.chromEnd_x hg38.knownCanonical.transcript  \
0                           12612              ENST00000456328.2   
1                           13220              ENST00000456328.2   
2                           15004              ENST00000488147.1   
3                           15795              ENST00000488147.1   
4                           16606              ENST00000488147.1   

   transcriptno direction #hg38.knownCanonical.chrom_y  \
0             0         +                         chr1   
1             1         +                         chr1   
2             0         -                         chr1   
3             1         -                         chr1   
4             2         -                         chr1   

   hg38.knownCanonical.chromStart_y  hg38.knownCanonical.chromEnd_y  \
0                             11868                           14409   
1                             11868                           14409   
2                             14403                           29570   
3                             14403                           29570   
4                             14403                           29570   

  hg38.kgXref.geneSymbol  feature_size  totalbp_forcorrespondingTranscript  \
0                DDX11L1           385                                  -1   
1                DDX11L1           499                                  -1   
2                 WASH7P           503                                  -1   
3                 WASH7P           757                                  -1   
4                 WASH7P           659                                  -1   

   bp_percentage  bp_inscaled_block  startingtranNumber  
0             -1                 -1                  -1  
1             -1                 -1                  -1  
2             -1                 -1                  -1  
3             -1                 -1                  -1  
4             -1                 -1                  -1

In [4]:
grtranscript=indf.groupby(['hg38.knownCanonical.transcript'])

for name, group in grtranscript:
    totalbp=group['feature_size'].sum()
    indf.loc[group.index,'totalbp_forcorrespondingTranscript']=totalbp
indf.head()

#hg38.knownCanonical.chrom_x  hg38.knownCanonical.chromStart_x  \
0                         chr1                             12227   
1                         chr1                             12721   
2                         chr1                             14501   
3                         chr1                             15038   
4                         chr1                             15947   

   hg38.knownCanonical.chromEnd_x hg38.knownCanonical.transcript  \
0                           12612              ENST00000456328.2   
1                           13220              ENST00000456328.2   
2                           15004              ENST00000488147.1   
3                           15795              ENST00000488147.1   
4                           16606              ENST00000488147.1   

   transcriptno direction #hg38.knownCanonical.chrom_y  \
0             0         +                         chr1   
1             1         +                         chr1   
2             0         -                         chr1   
3             1         -                         chr1   
4             2         -                         chr1   

   hg38.knownCanonical.chromStart_y  hg38.knownCanonical.chromEnd_y  \
0                             11868                           14409   
1                             11868                           14409   
2                             14403                           29570   
3                             14403                           29570   
4                             14403                           29570   

  hg38.kgXref.geneSymbol  feature_size  totalbp_forcorrespondingTranscript  \
0                DDX11L1           385                                 884   
1                DDX11L1           499                                 884   
2                 WASH7P           503                               13816   
3                 WASH7P           757                               13816   
4                 WASH7P           659                               13816   

   bp_percentage  bp_inscaled_block  startingtranNumber  
0             -1                 -1                  -1  
1             -1                 -1                  -1  
2             -1                 -1                  -1  
3             -1                 -1                  -1  
4             -1                 -1                  -1

In [5]:
indf['bp_percentage']=indf['feature_size']/indf['totalbp_forcorrespondingTranscript']*100
indf['bp_percentage']=indf['bp_percentage'].round()


indf.to_csv(outfile+"_likelog_first.txt",sep="\t",index=False)
indf.head()

#hg38.knownCanonical.chrom_x  hg38.knownCanonical.chromStart_x  \
0                         chr1                             12227   
1                         chr1                             12721   
2                         chr1                             14501   
3                         chr1                             15038   
4                         chr1                             15947   

   hg38.knownCanonical.chromEnd_x hg38.knownCanonical.transcript  \
0                           12612              ENST00000456328.2   
1                           13220              ENST00000456328.2   
2                           15004              ENST00000488147.1   
3                           15795              ENST00000488147.1   
4                           16606              ENST00000488147.1   

   transcriptno direction #hg38.knownCanonical.chrom_y  \
0             0         +                         chr1   
1             1         +                         chr1   
2             0         -                         chr1   
3             1         -                         chr1   
4             2         -                         chr1   

   hg38.knownCanonical.chromStart_y  hg38.knownCanonical.chromEnd_y  \
0                             11868                           14409   
1                             11868                           14409   
2                             14403                           29570   
3                             14403                           29570   
4                             14403                           29570   

  hg38.kgXref.geneSymbol  feature_size  totalbp_forcorrespondingTranscript  \
0                DDX11L1           385                                 884   
1                DDX11L1           499                                 884   
2                 WASH7P           503                               13816   
3                 WASH7P           757                               13816   
4                 WASH7P           659                               13816   

   bp_percentage  bp_inscaled_block  startingtranNumber  
0           44.0                 -1                  -1  
1           56.0                 -1                  -1  
2            4.0                 -1                  -1  
3            5.0                 -1                  -1  
4            5.0                 -1                  -1

In [ ]:

indf.rename(columns={'bp_percentage':'orig_bp_percentage'},inplace=True)

indf['bp_percentage']=indf['orig_bp_percentage'].copy()
indf.loc[indf['bp_percentage']==0,'bp_percentage']=1

sanitygrhundread=indf.groupby(['hg38.knownCanonical.transcript'])
for name, sanitygrhundreadgroup in sanitygrhundread:
    allpeercentsanity=sanitygrhundreadgroup['bp_percentage'].sum()
    if allpeercentsanity!=100:
        fl=0
        while allpeercentsanity>100:
            gap=int(allpeercentsanity)-100
            sortedgr=indf[indf['hg38.knownCanonical.transcript']==name].sort_values(by=['bp_percentage'],ascending=False)
            
            ###################
            sortedgr=sortedgr[sortedgr['bp_percentage']>1]
            ###################
            
            sortedgrhead=sortedgr.head(n=gap)
            
            indf.loc[sortedgrhead.index,'bp_percentage']=indf.loc[sortedgrhead.index,'bp_percentage']-1
            
            
            ##################
            allpeercentsanity=indf.loc[indf['hg38.knownCanonical.transcript']==name,'bp_percentage'].sum()
            if allpeercentsanity==100:
                fl=1
             
            ###################
        if fl==1:
            continue

        while allpeercentsanity<100:
            gap=100-int(allpeercentsanity)
            sortedgr=indf[indf['hg38.knownCanonical.transcript']==name].sort_values(by=['bp_percentage'])
        
            ###################
            sortedgr=sortedgr[sortedgr['bp_percentage']<99]
            sortedgr=sortedgr[sortedgr['orig_bp_percentage']>0]
            ###################
            sortedgrhead=sortedgr.head(n=gap)
        
            indf.loc[sortedgrhead.index,'bp_percentage']=indf.loc[sortedgrhead.index,'bp_percentage']+1
            
            ##################
            allpeercentsanity=indf.loc[indf['hg38.knownCanonical.transcript']==name,'bp_percentage'].sum()
            if allpeercentsanity==100:
                break
            ###################
    
    
sanitygrhundread=indf.groupby(['hg38.knownCanonical.transcript'])
for name, sanitygrhundreadgr2 in sanitygrhundread:
    allpeercentsanity2=sanitygrhundreadgr2['bp_percentage'].sum()
    if allpeercentsanity2!=100:
        print(name,allpeercentsanity2)
        sys.exit(1)
        
indf['bp_inscaled_block']=indf['feature_size']/indf['bp_percentage'] 
indf.to_csv(outfile+"_likelog_second.txt",sep="\t",index=False)

In [ ]:
def populate_rows(arow):
    every_MAXblock_size=math.ceil(arow['bp_inscaled_block'])
    every_MINblock_size=math.floor(arow['bp_inscaled_block'])
    totalblocks=arow['bp_percentage']
    
    adjustedMAXtotalbp=every_MAXblock_size*totalblocks
    
    adjustedMINtotalbp=every_MINblock_size*totalblocks
    
    bpadded=adjustedMAXtotalbp-arow['feature_size']
    bpdiscarded=arow['feature_size']-adjustedMINtotalbp
    
    MAXblock_start=arow['hg38.knownCanonical.chromStart_x']-math.ceil(bpadded/2)
    if MAXblock_start<0:
        MAXblock_start=0
    
    MAXblockend=arow['hg38.knownCanonical.chromEnd_x']+math.floor(bpadded/2)
    
    
    MINblockstart=arow['hg38.knownCanonical.chromStart_x']+math.floor(bpdiscarded/2)
    MINblockend=arow['hg38.knownCanonical.chromEnd_x']-math.ceil(bpdiscarded/2)
    
    

    
    if MINblockstart>=MINblockend:
        print("whatttttttt?")
        sys.exit(1)
    
    if (MAXblockend-MAXblock_start)/every_MAXblock_size!=totalblocks:
        print(arow)
        print("whatttttttt MAXblocks?")
        sys.exit(1)
    
    if (MINblockend-MINblockstart)/every_MINblock_size!=totalblocks:
        print(arow)
        print("whatttttttt MINblocks?")
        sys.exit(1)
    
    
    #print(bpadded)
    #print(bpdiscarded)
    
    #print(MAXblock_start,MAXblockend,(MAXblockend-MAXblock_start)/every_MAXblock_size)
    
    MAXblockdf=justpopulation(arow['#hg38.knownCanonical.chrom_x'],MAXblock_start,MAXblockend,every_MAXblock_size,totalblocks,arow['hg38.knownCanonical.transcript'],arow['startingtranNumber'])
   

    #print(MINblockstart,MINblockend,(MINblockend-MINblockstart)/every_MINblock_size)
    
    MINblockdf=justpopulation(arow['#hg38.knownCanonical.chrom_x'],MINblockstart,MINblockend,every_MINblock_size,totalblocks,arow['hg38.knownCanonical.transcript'],arow['startingtranNumber'])
    #print(MINblockdf)
    
    return MAXblockdf,MINblockdf

    
def justpopulation(blockchr,blockstart,blockend,blocksize,howmany,transcript,startblocknumber):
    outdict = defaultdict(list)
    currentstart=blockstart
    blockcoungter=startblocknumber
    while currentstart<blockend:
        currentend=currentstart+blocksize
        outdict['#hg38.knownCanonical.chrom_x'].append(blockchr)
        outdict['hg38.knownCanonical.chromStart_x'].append(currentstart)
        outdict['hg38.knownCanonical.chromEnd_x'].append(currentend)
        outdict['hg38.knownCanonical.transcript'].append(transcript)
        outdict['transcriptno'].append(blockcoungter)
        
        
        blockcoungter=blockcoungter+1
        currentstart=currentend
        
        
    if currentstart!=blockend:
        print(currentstart,blockcoungter)
        print(blockchr,blockstart,blockend,blocksize,howmany)
        print("justpopulation, currentstart!=blockend")
        sys.exit(1)
    
    if blockcoungter!=(howmany+startblocknumber):
        print(blockchr,blockstart,blockend,blocksize,howmany)
        print("justpopulation, blockcoungter!=howmany")
        sys.exit(1)
    
    outdf=pd.DataFrame.from_dict(outdict)
    
    return outdf
        
    

    
    

In [ ]:
for index, row in indf.iterrows():
    if row['transcriptno']==0:
        indf.loc[index,'startingtranNumber']=0
    
    else:
        if indf.loc[index-1,'hg38.knownCanonical.transcript']!=row['hg38.knownCanonical.transcript']:
            print('some prob')
            sys.exit(1)
        
        if indf.loc[index-1,'hg38.knownCanonical.chromStart_x']>=row['hg38.knownCanonical.chromStart_x']:
            print("not sorted?")
            sys.exit(1)
        
        
        if indf.loc[index-1,'transcriptno']+1!=row['transcriptno']:
            print("my transcriptno assumption wrong?")
            sys.exit(1)
        
        prevstartingtranNumber=indf.loc[index-1,'startingtranNumber']
        
        if prevstartingtranNumber==-1:
            print("prevstartingtranNumber not set")
            sys.exit(1)
        
        prevtotalblock=indf.loc[index-1,'bp_percentage']
        
        nextstartnumber=prevstartingtranNumber+prevtotalblock
        indf.loc[index,'startingtranNumber']=nextstartnumber
indf.head()

In [ ]:
MAXblocklist=[]
MINblocklist=[]
for index, row in indf.iterrows():
     
        
    
    tmpMAXblock,tmpMINblock=populate_rows(row)
    
    MAXblocklist.append(tmpMAXblock)
    MINblocklist.append(tmpMINblock)

    
MAXmasterdf=pd.concat(MAXblocklist)
MINmasterdf=pd.concat(MINblocklist)




In [ ]:
MAXmasterdf.to_csv(outfile+"_MAX.txt",sep="\t",index=False)
MINmasterdf.to_csv(outfile+"_MIN.txt",sep="\t",index=False)